In [1]:
from transformers import (
    AutoModelForSequenceClassification, 
    AutoTokenizer,
    TextClassificationPipeline,
)
import torch

/home/vitor/.cache/pypoetry/virtualenvs/pln-trabalho-final-9al6VtFs-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
THRESHOLD = 0.5

In [3]:
MODEL_PATH = "CarusoVitor/multiemotion-portuguese"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
labels = ['Anger', 'Disgust', 'Fear', 'Joy', 'Sadness', 'Surprise', 'Neutral']
id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}

In [5]:
model = AutoModelForSequenceClassification.from_pretrained( 
    MODEL_PATH,
    problem_type="multi_label_classification",
    num_labels=len(labels),
    id2label=id2label,
    label2id=label2id
).to(device)
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)

OSError: CarusoVitor/multisentiment-portuguese is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

In [6]:
pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer, return_all_scores=True, function_to_apply="sigmoid")


Device set to use cpu
/home/vitor/.cache/pypoetry/virtualenvs/pln-trabalho-final-9al6VtFs-py3.10/lib/python3.10/site-packages/transformers/pipelines/text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [11]:
scores = pipe("Que tristeza")
predicted = [item["label"] for item in scores if item["score"] > THRESHOLD]

[[{'label': 'Anger', 'score': 0.25274601578712463},
  {'label': 'Disgust', 'score': 0.01847831718623638},
  {'label': 'Fear', 'score': 0.06113993749022484},
  {'label': 'Joy', 'score': 0.0670752003788948},
  {'label': 'Sadness', 'score': 0.17100153863430023},
  {'label': 'Surprise', 'score': 0.11274018883705139},
  {'label': 'Neutral', 'score': 0.3686459958553314}]]